<a href="https://colab.research.google.com/github/jiangzian96/DS1012/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3: Word Embeddings

In this homework assignment, we are going to tackle two different exercises involving word embeddings. First, we are going to explore using the semantic orientation method to score words. Second, we will explore the effect of context sizes in building embeddings based on word cooccurrences.

# Part 0: Setup

In [0]:
!pip install sacremoses

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import sacremoses
import tqdm
import time
from matplotlib import pyplot as plt
%matplotlib inline

We're going load a set of 50D word vectors from GloVe. If want to download other versions of GloVe, you can click [here](https://nlp.stanford.edu/projects/glove/).

In [0]:
!wget https://docs.google.com/uc?id=1si-rTb4nALWb-Wah5_tqbs9KXAwE3qhK -O glove.aa
!wget https://docs.google.com/uc?id=1_XywxYH56-tHepqxpUfE-Y6xNQSZiu-7 -O glove.ab
!wget https://docs.google.com/uc?id=1ISSLqQWL8EO8blT54RKahkDQhtmSp9ae -O glove.ac
!wget https://docs.google.com/uc?id=1qHqCKLFO0Zdyhmb_ctd63aaFRLNGiDfB -O glove.ad
!cat glove.?? > 'glove.6B.50d.txt'

We'll load the GloVe embeddings similar to previous labs. `words_to_load` specifies how many word vectors we want to load. The words are saved in frequency order, so specifying 50,000 means that we only want to work with the 50,000 most frequent words from the source corpus.

In [0]:
words_to_load = 50000

with open('./glove.6B.50d.txt') as f:
    loaded_embeddings = np.zeros((words_to_load, 50))
    words_to_index = {}
    ordered_words = []
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        
        s = line.split()
        loaded_embeddings[i, :] = np.asarray(s[1:])
        words_to_index[s[0]] = i
        ordered_words.append(s[0])
print("GloVe loaded")

Let's look at the embedding for the word "potato".

In [0]:
loaded_embeddings[words_to_index['potato']]

# Part 1: The Semantic Orientation Method [40]

The __semantic orientation__ method of [Turney and Littman 2003](http://doi.acm.org/10.1145/944012.944013) is a method for automatically scoring words along some single semantic dimension like sentiment. It works from a pair of small seed sets of words that represent two opposing points on that dimension.

*Some code in this section was adapted from Stanford CS 224U*

Here's a sample pair of seed sets:

In [0]:
positive_words = ['good', 'great', 'awesome', 'like', 'love']
negative_words = ['bad', 'awful', 'terrible', 'hate', 'dislike']

In [0]:
class SemanticOrientation:
    
    def __init__(self, seed_pos, seed_neg):
        """
        args: 
            - seed_pos: list of 'positive' words
            - seed_neg: list of 'negative' words
        """
        self.seed_pos = seed_pos
        self.seed_neg = seed_neg
        seed_pos_indices = [words_to_index[seed] for seed in seed_pos]
        seed_neg_indices = [words_to_index[seed] for seed in seed_neg]
        
        # Get matrices of embeddings for positive and negative words
        self.seed_pos_mat = loaded_embeddings[seed_pos_indices]
        self.seed_neg_mat = loaded_embeddings[seed_neg_indices]
        
    def determine_coefficient(self, candidate_words):
        """
        args:
            - candidate_words: single word, or list of words
        """
        
        # Optionally handle a single word
        one_word = isinstance(candidate_words, str)
        if one_word:
            candidate_words = [candidate_words]

        # Retrieve matrix of embeddings for candidate words
        candidates_mat = np.array([
            loaded_embeddings[words_to_index[candidate_word]]
            for candidate_word in candidate_words
        ])
        # Compute the cosine similarity between the seed word and 
        #   the candidate word embeddings, and average
        pos_sim = cosine_similarity(self.seed_pos_mat, candidates_mat).mean(axis=0)
        neg_sim = cosine_similarity(self.seed_neg_mat, candidates_mat).mean(axis=0)
        # Compute the difference between the average positive and negative similarities
        diff = pos_sim - neg_sim

        if one_word:
            return diff[0]
        else:
            return diff

In [0]:
sentiment_so = SemanticOrientation(
    seed_pos=positive_words,
    seed_neg=negative_words,
)

Let's get the coefficients of some words and see if it lines up with our intuition.

In [0]:
sentiment_so.determine_coefficient('abhorrent')

In [0]:
sentiment_so.determine_coefficient('excellent')

In [0]:
sentiment_so.determine_coefficient('amazing')

In [0]:
sentiment_so.determine_coefficient('okay')

In [0]:
sentiment_so.determine_coefficient('atrocious')

In [0]:
def scatter_and_annotate(ax, x_values, y_values, labels, color):
    ax.scatter(x_values, y_values, color=color)
    for i, label in enumerate(labels):
        ax.text(x_values[i], y_values[i], label, rotation=45)

def plot_semantic_orientation(semantic_orientation, words):
    fig = plt.figure(figsize=(8, 4))
    ax = fig.gca()
    x_values = semantic_orientation.determine_coefficient(semantic_orientation.seed_pos) 
    x_values = semantic_orientation.determine_coefficient(semantic_orientation.seed_pos) 

    scatter_and_annotate(
        ax=ax,
        x_values=semantic_orientation.determine_coefficient(semantic_orientation.seed_pos),
        y_values=[0] * len(semantic_orientation.seed_pos),
        labels=semantic_orientation.seed_pos,
        color="blue",
    )
    scatter_and_annotate(
        ax=ax,
        x_values=semantic_orientation.determine_coefficient(semantic_orientation.seed_neg),
        y_values=[0] * len(semantic_orientation.seed_neg),
        labels=semantic_orientation.seed_neg,
        color="red",
    )
    ax.axhline(0, color="gray")

    scatter_and_annotate(
        ax=ax,
        x_values=semantic_orientation.determine_coefficient(words),
        y_values=[-2] * len(words),
        labels=words,
        color="purple",
    )
    ax.axhline(-2, color="black")
    ax.set_ylim(-2.5, 1)
    ax.set_yticks([-2, 0])
    ax.set_yticklabels(["words", "seed"])

In [0]:
plot_semantic_orientation(
    semantic_orientation=sentiment_so, 
    words=["abhorrent", "excellent", "amazing", "okay", "atrocious"],
)

And sort our vocabulary by its score along the axis. For now, we're only scoring frequent words, since this process can be slow.

### Question 1.1 [10]

Using `sentiment_so`, compute the coefficients of all `ordered_words` and show the 10 most positive and 10 most negative words.

### Question 1.2 [30]

1. Create another `SemanticOrientation` object with a different set of contrasting "positive" and "negative" categories that are unrelated to sentiment (e.g solids vs. liquids, living things vs objects) and corresponding seed words. Have at least 5 words per category.
2. Pick another 5 words in either category and show their coefficients. Display them using the plotting code from above.
3. Using `ordered_words`, show the 10 most positive and 10 most "negative" words.
4. Pick 2 words that are unrelated to either category, compute their coefficients, and comment on them.

# Part 2: Exploring effect of context size [60 pts]

Let's take a step back and assume we are creating word embeddings from scratch.

We face many implicit and explicit design decisions in creating distributional word representations. In lecture, we discussed creating vectors using a co-occurence matrix built on neighboring pairs of words. We might suspect, however, that we can get more signal of word similarity by considering larger contexts than pairs of words.

We're going to download two files:

1. `sst.txt`, a list of sentences from the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/treebank.html).
2. `MTURK-771.csv`, a [word-relatedness dataset](http://www2.mta.ac.il/~gideon/mturk771.html).

In [0]:
!wget https://docs.google.com/uc?id=1uDpAm-eoZx-kS7UAPELfH0gGAxekkDVd -O sst.txt
!wget http://www2.mta.ac.il/~gideon/datasets/MTURK-771.csv -O MTURK-771.csv

In [0]:
mturk_path = "./MTURK-771.csv"
sst_path = "./sst.txt"

First, we will load the SST data.

In [0]:
def load_sst(sst_path):
    tokenizer = sacremoses.MosesTokenizer()
    results = []
    for line in tqdm.tqdm_notebook(pd.read_csv(sst_path, sep="\t")["sentence"].values):
        results.append(tokenizer.tokenize(line))
    return results
data = load_sst(sst_path)

### Question 2.1 [45]

Implement:
* `get_token_frequencies_and_cooccurrences`
* `prune_vocabulary`
* `build_cooccurrence_mat_from_counts`

Each of these will be used in  `build_cooccurrence_matrix`, which generates the co-occurence matrix for a window of arbitrary size and for the vocabulary of `max_vocab_size` most frequent words. 

In [0]:
def build_cooccurrence_matrix(data, max_vocab_size=20000, context_size=2, verbose=True):
    """ 
    args:
        - data: iterable where each item is a string sentence
        - max_vocab_size: maximum vocabulary size
        
    returns:
        - coocur_mat: co-occurrence matrix as a numpy array
    """
    if verbose:
        print("Counting words...")
    start_time = time.time()
    tok2freq, cooccur_counts = get_token_frequencies_and_cooccurrences(data, context_size)
    if verbose:
        print("\tFinished counting %d words in %.5f" % (len(tok2freq), time.time() - start_time))
        print("Pruning vocabulary...")
    tok2idx, idx2tok = prune_vocabulary(tok2freq, max_vocab_size)
    start_time = time.time()
    if verbose:
        print("\tFinished pruning vocabulary to %d words in %.5f" % (len(tok2idx), time.time() - start_time))
        print("Building co-occurrence matrix...")
    start_time = time.time()
    cooccur_mat = build_cooccurrence_mat_from_counts(idx2tok, cooccur_counts)
    if verbose:
        print("\tFinished building co-occurrence matrix in %.5f" % (time.time() - start_time))
    return cooccur_mat, tok2idx, idx2tok, cooccur_counts

In [0]:
from collections import defaultdict

def get_token_frequencies_and_cooccurrences(data, context_size):
    """ 
    Compute token frequencies and cooccurrences from data

    args: 
        - data: a list of list of words (or tokens)
        - context_size: the vocabulary size to prune to
    
    return:
        A tuple containing:
        - tok2freq: a dictionary mapping from words to count
        - cooccur_counts: a dictionary of dictionaries, where

              cooccur_counts[word1][word2]

          is how often word1 cooccurs with word2 within a given context_size
    """
    tok2freq = defaultdict(int)
    cooccur_counts = defaultdict(lambda: defaultdict(int))

    for datum in data:
        for i, tok in enumerate(datum):
            tok2freq[tok] += 1
            for k in range(context_size + 1):
                # === Exercise Start [1/3] === #
                raise NotImplementedError()
                # === Exercise End [1/3] === #
    return tok2freq, cooccur_counts

def prune_vocabulary(tok2freq, max_vocab_size):
    """ 
    Prune vocab by taking max_vocab_size most frequent words 

    args: 
        - tok2freq: a dictionary mapping from words to count
        - max_vocab_size: the vocabulary size to prune to
    
    return:
        A tuple containing:
        - tok2idx: a dictionary mapping from words to the index in idx2tok
        - idx2tok: a list of words (the vocabulary), of length max_vocab_size
    """
    tok_and_freqs = [(k, v) for k, v in tok2freq.items()]
    # === Exercise Start [2/3] === #
    raise NotImplementedError()
    # === Exercise End [2/3] === #
    return tok2idx, idx2tok

def build_cooccurrence_mat_from_counts(idx2tok, cooccur_counts):
    """ Build a cooccurrence matrix from counts and a vocab
    args:
        - idx2tok: a list of words (the vocabulary), of length N
        - cooccur_counts: a dictionary of dictionaries, where

              cooccur_counts[word1][word2]

          is how often word1 cooccurs with word2
    return:
        - mat: an NxN matrix (symmetric) of the occurrence counts of the
               words in idx2tok
    """
    vocab_size = len(idx2tok)
    mat = np.zeros([vocab_size, vocab_size])
    for i in tqdm.tqdm_notebook(range(vocab_size)):
        for j in range(i, vocab_size):
            # === Exercise Start [3/3] === #
            raise NotImplementedError()
            # === Exercise End [3/3] === #
    return mat

Let's build a cooccurrence matrix with `context_size=2`.

In [0]:
mat, tok2idx, idx2tok, _ = build_cooccurrence_matrix(data, max_vocab_size=10000, context_size=2)

In [0]:
tok1 = "the"
tok2 = "end"
print("'{}' and '{}' co-occur {} times".format(tok1, tok2, mat[tok2idx[tok1]][tok2idx[tok2]]))

### Question 2.2 [15]

We are going to evaluate the effect of varying context size in $\{1, 2, 3, 4\}$ (leaving all the other settings the same) on the quality of the learned word embeddings, as measured by performance (Spearman correlation) on the word similarity dataset [MTurk-771](http://www2.mta.ac.il/~gideon/mturk771.html) between human judgments and cosine similarity of the learned word vectors (the coccurrence matrices).

In this exercise, you will fill in parts of the code in the `evaluate_word_similarity` function. (Hint: make use of the imports we include at the top of the code block; the solution should be pretty short!)

In [0]:
# Hint: some helpful imports!
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity


def load_word_similarity_dataset(data_file):
    mturk_df = pd.read_csv(mturk_path, names=["word1", "word2", "similarity"])
    return mturk_df


def evaluate_word_similarity(mturk_df, mat, tok2idx):
    """ Evaluate the word embeddings by comparing the word similarities 
        to the word similarity scores from the MTurk-771 data.
    Notes:
        - In some cases, the words in the MTurk-771 data may not appear in 
          our vocabulary. In those cases, we will simpy skip over those
          MTurk-771 examples.
        - Use cosine_similarity to compute word similarities, and use
          spearmanr to compute the overall rank correlation.

    args:
        - mturk_df: DataFrame of MTurk-711 data
        - mat: cooccurrence matrix computed from above. These will serve as 
               our word embeddings.
        - idx2tok: a list of words (the vocabulary), of length N

    return:
        - rho: the rank correlation between the predicted word similarities
               and the ground truth word similarities from MTurk-771
    """
    preds = []
    trgs = []
    n_exs = 0
    for row in mturk_df.itertuples():
        if row.word1 in tok2idx and row.word2 in tok2idx:
            # === Exercise Start [1/2] === #
            pred_sim = NotImplemented
            # === Exercise End [1/2] === #
            preds.append(pred_sim)
            trgs.append(row.similarity)
            n_exs += 1

    # === Exercise Start [2/2] === #
    rho = NotImplemented
    # === Exercise End [2/2] === #
    print("Evaluated on %d of %d examples" % (n_exs, len(mturk_df)))
    return rho

In [0]:
mturk_df = load_word_similarity_dataset(mturk_path)
scores = []
context_sizes = [1, 2, 3, 4]
for context_size in context_sizes:
    mat, tok2idx, idx2tok, _ = build_cooccurrence_matrix(data, max_vocab_size=10000, context_size=context_size)
    score = evaluate_word_similarity(mturk_df, mat, tok2idx)
    scores.append(score)
print(scores)

In [0]:
score = evaluate_word_similarity(mturk_df, mat, tok2idx)

In [0]:
plt.plot(context_sizes, scores, marker='o')
plt.xlabel("Context size")
plt.ylabel("Word similarity correlation")